In [1]:
!ls /l/disk1/awstebas/data/pretrained_models/modelo_acustico/espnet2/CPQD-22050-fastspeech2/MU_CPQD_xvector_2.0/train_lists/

cpqd-ptbr-es-MU-fastspeech_dev.txt   cpqd-ptbr-es-MU-fastspeech_leiame.txt
cpqd-ptbr-es-MU-fastspeech_eval.txt  cpqd-ptbr-es-MU-fastspeech_train.txt


In [2]:
# The path where getting the training files, from 6 speakers with around 24 hours of speech
train_list_path = '/l/disk1/awstebas/data/pretrained_models/modelo_acustico/espnet2/CPQD-22050-fastspeech2/MU_CPQD_xvector_2.0/train_lists/'
file_train = 'cpqd-ptbr-es-MU-fastspeech_train.txt'
file_val = 'cpqd-ptbr-es-MU-fastspeech_dev.txt'
file_test = 'cpqd-ptbr-es-MU-fastspeech_eval.txt'

In [3]:
import os
import sys

file_path_train = os.path.join(train_list_path, file_train)
file_path_val = os.path.join(train_list_path, file_val)
file_path_test = os.path.join(train_list_path, file_test)
file_path_train

'/l/disk1/awstebas/data/pretrained_models/modelo_acustico/espnet2/CPQD-22050-fastspeech2/MU_CPQD_xvector_2.0/train_lists/cpqd-ptbr-es-MU-fastspeech_train.txt'

In [4]:
with open(file_path_train) as f:
    for line in f.readlines():
        print(line.split('	'))
        break

['adriana', 'pt_BR', 'data/TTS/speaker-adriana/aec/wav22/aec0001.wav', 'Você quer agendar seus exames?\n']


# Lets process the text to get phones

In [1]:
import os
def get_phones(txt):
    
    with open('tmp.txt', "w", encoding="utf-8") as f:
        f.writelines(txt)
    
    cmd = '/workspace/tool_language/tool_language -l /workspace/tool_language/libptbr.so.4.6.0 -p /workspace/tool_language/ -i tmp.txt -o tmp_pnh.txt --phonemes -s'
    
    os.system(cmd)
    
    with open ('tmp_pnh.txt', "r") as f:
        lines = f.readlines()
    
    return lines[0][:-1]

In [3]:
txt = 'Este é um exemplo de síntese gerada pelo meu último modelo.'

get_phones(txt)

"#p 'ee ss ts ic 'eh un ee zz 'en pp ll uc dz ic ss 'in tt ee zz ic zh ee rr 'aa dd ac pp 'ee ll uc mm 'ee uw 'uu uw ts ic mm uc mm oo dd 'ee ll uc #p"

In [9]:
## TRAIN PROCESS

from tqdm import tqdm
import pandas as pd

wavs_p = []
speaker_p = []
text_p = []
phones_p = []

tts_data_path = '/l/disk1/awstebas/'

with open(file_path_train) as f:
    for line in tqdm(f.readlines()):
        
        splitted = line.split('\t')
        
        if(splitted[1] == 'pt_BR'):
        
            w_path = os.path.join(tts_data_path,splitted[2])
            if(os.path.isfile(w_path)):
                wavs_p.append(w_path) 
            else:
                print(f'ERROR! {w_path} does not exit!')
                break
            speaker_p.append(splitted[0])

            text_p.append(splitted[-1])
            phones_p.append(get_phones(splitted[-1]))
            
df_train = pd.DataFrame({'norm_text': text_p,
                   'phonetic_transcription': phones_p,
                   'wav_path': wavs_p,
                   'speaker': speaker_p})

df_train.head(), df_train.shape

100%|██████████| 61041/61041 [57:01<00:00, 17.84it/s]  


(                                           norm_text  \
 0                   Você quer agendar seus exames?\n   
 1            Você quer agendar exames para quando?\n   
 2  Para quando você gostaria de agendar seus exam...   
 3                 Você pode reagendar seus exames.\n   
 4               Gostaria de reagendar seus exames?\n   
 
                               phonetic_transcription  \
 0  #p vv oo ss 'ee kk 'eh rr aa zh en dd 'aa rf s...   
 1  #p vv oo ss 'ee kk 'eh rr aa zh en dd 'aa rr e...   
 2  #p pp 'aa rr ac kk uw 'an dd uc vv oo ss 'ee g...   
 3  #p vv oo ss 'ee pp 'oh dz ic rx ee aa zh en dd...   
 4  #p gg oo ss tt aa rr 'ii ac dz ic rx ee aa zh ...   
 
                                             wav_path  speaker  
 0  /l/disk1/awstebas/data/TTS/speaker-adriana/aec...  adriana  
 1  /l/disk1/awstebas/data/TTS/speaker-adriana/aec...  adriana  
 2  /l/disk1/awstebas/data/TTS/speaker-adriana/aec...  adriana  
 3  /l/disk1/awstebas/data/TTS/speaker-adriana/aec..

In [10]:
## VAL PROCESS

wavs_p = []
speaker_p = []
text_p = []
phones_p = []

tts_data_path = '/l/disk1/awstebas/'

with open(file_path_val) as f:
    for line in tqdm(f.readlines()):
        
        splitted = line.split('\t')
        
        if(splitted[1] == 'pt_BR'):
            
            w_path = os.path.join(tts_data_path,splitted[2])
            if(os.path.isfile(w_path)):
                wavs_p.append(w_path) 
            else:
                print(f'ERROR! {w_path} does not exit!')
                break
            speaker_p.append(splitted[0])

            text_p.append(splitted[-1])
            phones_p.append(get_phones(splitted[-1]))

df_val = pd.DataFrame({'norm_text': text_p,
                   'phonetic_transcription': phones_p,
                   'wav_path': wavs_p,
                   'speaker': speaker_p})

df_val.head()

100%|██████████| 400/400 [00:24<00:00, 16.66it/s]


,norm_text,phonetic_transcription,wav_path,speaker
0,"E sim chegar ao local, se identificar, e parti...",#p ic ss 'in sh ee gg 'aa rr 'aa uw ll oo kk '...,/l/disk1/awstebas/data/TTS/speaker-adriana/riq...,adriana
1,Tem que ser o maior dos últimos cinco anos.\n,#p tt 'en kk ic ss 'ee rr uc mm aa ij 'oh rf d...,/l/disk1/awstebas/data/TTS/speaker-adriana/riq...,adriana
2,"No campus da PUC, veio ao chão no domingo.\n",#p nn uc kk 'an pp uc zz dd ac pp 'uu kk ic #c...,/l/disk1/awstebas/data/TTS/speaker-adriana/riq...,adriana
3,Ouvir o autor sobre o ensino de adultos.\n,#p oo uw vv 'ii rr uc aa uw tt 'oo rf ss 'oo b...,/l/disk1/awstebas/data/TTS/speaker-adriana/riq...,adriana
4,"Quadrilha de Higienópolis, com veículos envolv...",#p kk uw aa dd rd 'ii lh ac dz ic ii zh ii ee ...,/l/disk1/awstebas/data/TTS/speaker-adriana/riq...,adriana


In [11]:
## TEST PROCESS

wavs_p = []
speaker_p = []
text_p = []
phones_p = []

tts_data_path = '/l/disk1/awstebas/'

with open(file_path_test) as f:
    for line in tqdm(f.readlines()):
        
        splitted = line.split('\t')
        
        if(splitted[1] == 'pt_BR'):
        
            w_path = os.path.join(tts_data_path,splitted[2])
            if(os.path.isfile(w_path)):
                wavs_p.append(w_path) 
            else:
                print(f'ERROR! {w_path} does not exit!')
                break
            speaker_p.append(splitted[0])

            text_p.append(splitted[-1])
            phones_p.append(get_phones(splitted[-1]))

df_test = pd.DataFrame({'norm_text': text_p,
                   'phonetic_transcription': phones_p,
                   'wav_path': wavs_p,
                   'speaker': speaker_p})

df_test.head()

100%|██████████| 200/200 [00:11<00:00, 16.89it/s]


,norm_text,phonetic_transcription,wav_path,speaker
0,Conhecia o trânsito naquela altura da pista.\n,#p kk on nh ee ss 'ii ac uc tt rd 'an zz ic tt...,/l/disk1/awstebas/data/TTS/speaker-adriana/riq...,adriana
1,Sua leitura do centro cultural deste nosso paí...,#p ss 'uu ac ll ee ij tt 'uu rr ac dd uc ss 'e...,/l/disk1/awstebas/data/TTS/speaker-adriana/riq...,adriana
2,"Ao vivo às quartas e no sábado, há troca de al...",#p 'aa uw vv 'ii vv uc 'aa ss kk uw 'aa rf tt ...,/l/disk1/awstebas/data/TTS/speaker-adriana/riq...,adriana
3,Lugar na região para quem for sócio da empresa.\n,#p ll uu gg 'aa rf nn ac rx ee zh ii 'an wn pp...,/l/disk1/awstebas/data/TTS/speaker-adriana/riq...,adriana
4,Compania lidera pesquisas feita até as oito e ...,#p kk on pp an nn 'ii ac ll ii dd 'eh rr ac pp...,/l/disk1/awstebas/data/TTS/speaker-adriana/riq...,adriana


# REMOVING STYLISH DATA

In [30]:
def get_rec(wpath):
    return wpath.split('/')[7]

recs = []
for w in df_train.wav_path:
    recs.append(get_rec(w))
df_train['rec'] = recs

recs = []
for w in df_val.wav_path:
    recs.append(get_rec(w))
df_val['rec'] = recs

recs = []
for w in df_test.wav_path:
    recs.append(get_rec(w))
df_test['rec'] = recs

In [34]:
df_test['rec'].unique() # Only in train we have style speech

array(['riqueza_fonetica', 'textofala', 'rf_selecionadas'], dtype=object)

In [35]:
to_exclude = []

for r in df_train.rec.unique():
    if('eps_' in r):
        to_exclude.append(r)
to_exclude

['eps_acolhedor_aco',
 'eps_acolhedor_ani',
 'eps_acolhedor_gen',
 'eps_acolhedor_rf',
 'eps_animado_aco',
 'eps_animado_ani',
 'eps_animado_gen',
 'eps_animado_rf',
 'eps_neutro_aco',
 'eps_neutro_ani',
 'eps_neutro_gen',
 'eps_neutro_rf',
 'eps_rispido_aco',
 'eps_rispido_acorf',
 'eps_rispido_gen',
 'eps_rispido_ris']

In [39]:
df_train.shape, df_train[~df_train['rec'].isin(to_exclude)].shape, df_train[df_train['rec'].isin(to_exclude)].shape

((44902, 6), (39722, 6), (5180, 6))

In [40]:
## FILTERING TRAIN
df_train = df_train[~df_train['rec'].isin(to_exclude)]

# Checking volumetries

In [41]:
import librosa

def get_durations(wpath):
    y, sr = librosa.load(wpath, sr = None)
    return len(y)/sr

In [42]:
durs = []
for w in df_train['wav_path']:
    durs.append(get_durations(w))
df_train['durs'] = durs

durs = []
for w in df_val['wav_path']:
    durs.append(get_durations(w))
df_val['durs'] = durs

durs = []
for w in df_test['wav_path']:
    durs.append(get_durations(w))
df_test['durs'] = durs

<ipython-input-42-49c4d5cd881d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['durs'] = durs


In [43]:
df_train.durs.sum()/3600, df_val.durs.sum()/3600, df_test.durs.sum()/3600

(38.74586798941799, 0.35842800453514745, 0.17809230284706473)

In [44]:
# df_train per speaker
df_train.groupby('speaker').agg({'durs': 'sum'})/3600

,durs
speaker,
adriana,10.605199
arianne,1.078933
chiquinho,4.551724
claudia,8.934560
edgard,6.348841
rosana,7.226612


In [45]:
# df_val per speaker
df_val.groupby('speaker').agg({'durs': 'sum'})/3600

,durs
speaker,
adriana,0.109410
arianne,0.006608
chiquinho,0.055547
claudia,0.083987
edgard,0.049193
rosana,0.053682


In [46]:
# df_test per speaker
df_test.groupby('speaker').agg({'durs': 'sum'})/3600

,durs
speaker,
adriana,0.050233
arianne,0.002969
chiquinho,0.027331
claudia,0.043346
edgard,0.028027
rosana,0.026187


# Saving metafiles

In [47]:
# Finally, lets save files as meta_data format for cpqd_read, which use ';' as separator

cols = ['phonetic_transcription', 'wav_path', 'speaker']

df_train[cols].to_csv('universal_cpqd_train.csv', index = False, sep = ';', encoding='utf-8')
df_val[cols].to_csv('universal_cpqd_val.csv', index = False, sep=';', encoding = 'utf-8')
df_test[cols].to_csv('universal_cpqd_test.csv', index = False, sep=';', encoding = 'utf-8')

# Just checking why adriana has so much data

In [23]:
def get_rec(wpath):
    return wpath.split('/')[7]

In [24]:
recs = []

for w in df_train.wav_path:
    recs.append(get_rec(w))

In [28]:
pd.DataFrame({'recs': recs}).recs.unique()

array(['aec', 'akiva', 'akiva_fidelizacao', 'akiva_portabilidade',
       'aplicacao', 'atendimento', 'avm', 'avm_custom', 'bradesco',
       'bradesco_bia', 'bradesco_ura', 'callcenter', 'chat_covid',
       'comemorativas', 'datas', 'digitos', 'dnk_expr',
       'elev_dentalprime', 'emocao', 'empresas', 'enderecos',
       'eps_acolhedor_aco', 'eps_acolhedor_ani', 'eps_acolhedor_gen',
       'eps_acolhedor_rf', 'eps_animado_aco', 'eps_animado_ani',
       'eps_animado_gen', 'eps_animado_rf', 'eps_neutro_aco',
       'eps_neutro_ani', 'eps_neutro_gen', 'eps_neutro_rf',
       'eps_rispido_aco', 'eps_rispido_acorf', 'eps_rispido_gen',
       'eps_rispido_ris', 'freqpalavras', 'horas', 'ibm', 'ibm_fastshop',
       'ibm_tim', 'interrogativas', 'interrogativas_ou',
       'interrogativas_qu', 'interrogativas_sn', 'invenit',
       'marketing_cpqd', 'neutro', 'nomes', 'nomes_freq', 'nomes_int',
       'numeros', 'pg_amc', 'pg_americo', 'pg_avon', 'pg_demoKami_v2',
       'pg_demoPrioritar